<a href="https://colab.research.google.com/github/margaretfoster/IRTM-Synth/blob/master/EuroB_Subset_Synth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following the quickstart:
https://docs.synthetic.ydata.ai/1.3/getting-started/quickstart/#synthesizing-a-tabular-dataset

Install the package:

In [1]:
 #!pip install ydata-synthetic==1.1.0

Authenticate:

In [2]:
import os
import pandas as pd
import io
from google.colab import files #to load local data
from google.colab import drive

import ydata_synthetic
from ydata_synthetic.synthesizers.regular import RegularSynthesizer #for tabular
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters

drive.mount('/content/drive')

Mounted at /content/drive


Authentication token:

In [19]:

token = 'your_token_here'
os.environ["YDATA_TOKEN"] = token

Load data, previously uploaded to Google Drive:

In [ ]:
#uploaded = files.upload()



Saving eurob_toy.csv to eurob_toy (1).csv


In [4]:
#data_url ="https://docs.google.com/spreadsheets/d/1sxw6tYkPYY66jKdSegJhLQvVL6pDsg79OZF8DmRXwaA/edit?usp=sharing"
#https://docs.google.com/spreadsheets/d/1-6MRByoVZUp7ZC8qDT3CV5jU709HpLupwX1L2IJzcfU/edit?usp=sharing
#eurob_toy = pd.read_csv(data_url, on_bad_lines='warn') ## skips a lot of rows for unknown reason


data_path = '/content/drive/MyDrive/Synthdata/'
eurob_toy = pd.read_csv(data_path + 'eurob_toy.csv') ## loads as expected

print(eurob_toy.shape) ## 3872, 360; 3872 is 10% of the original survey & 360 is questions + metadata





(3872, 367)


Determine NA structure:

In [5]:
nas = eurob_toy.isnull().sum(axis = 0)



In [6]:
#print(nas[nas > 0].sort_values(ascending=False)) #212 columns with some missing info

print(len(nas[nas > 1000])) ## 162 columns with over > 1000 missing

## Check what the columns are, and drop them
##

nas_thousand = list(nas[nas > 1000].index)
print(nas_thousand) # check the col

# NA strategy: impute for less than 25% missing, remove columns with more missingness.

nas_quarter= list(nas[nas >= (len(nas)*.25)].index)
print(len(nas_quarter)) #195


162
['qb6_4', 'qa5_15', 'qa18_4', 'qb5_11', 'qd4b_4', 'qd3_2', 'qa20_8', 'qa9_3', 'qb2_2', 'qb3_4', 'qd4b_5', 'qa5_3', 'qb5_6', 'qb6_7', 'qa6b_12', 'qd6_2', 'qb5_5', 'qc4_1', 'qd4t_1', 'qd4t_4', 'qb3_2', 'qa11_2', 'qb5_9', 'qb5_3', 'qb1_1', 'qd3_6', 'qc4_5', 'qa9_2', 'qa20_6', 'qa15', 'qb6_1', 'qa7_4', 'qa6b_11', 'qd4a', 'qb2_1', 'qa5_2', 'qb5_12', 'qd4t_3', 'qa5_1', 'qb2_3', 'qd6_4', 'qb7_2', 'qa20_3', 'qd6_1', 'qc4_16', 'qd3_3', 'qb3_3', 'qc2_1', 'qa11_8', 'qb4_3', 'qc4_17', 'qa11_6', 'qa6e', 'qd7_4', 'qa20_10', 'qb6_3', 'qd4b_2', 'qa11_12', 'qc4_18', 'qd6_3', 'qa5_9', 'qc4_9', 'qd3_4', 'qd7_2', 'qb4_5', 'qa5_7', 'qb4_2', 'qb5_8', 'qb5_1', 'qa6f', 'qa6b_7', 'qb5_10', 'qd7_3', 'qd4t_5', 'qc4_2', 'qc4_4', 'qa5_12', 'qb2_4', 'qa20_2', 'qc4_8', 'qd4b_6', 'qa5_14', 'qa11_9', 'qb5_2', 'qa5_5', 'qc4_15', 'qc4_11', 'qc2_3', 'qa20_7', 'qb5_7', 'qa11_13', 'qd4b_8', 'qa11_4', 'qa11_7', 'qd4t_6', 'qc6_3', 'qc4_3', 'qb1_3', 'qb5_4', 'qa20_4', 'qc5_2', 'qd4t_8', 'qa20_9', 'qa8_3', 'qa8_4', 'qa11_1

At this point, in a real analysis, I would want to do some diagnostics about what distributions change after I remove the columns. Do we systematically lose countries, important questions, etc.

For this proof-of-concept, I'll just do it though.

---



In [7]:
ebt = eurob_toy.drop(columns= nas_quarter)

print(ebt.shape) ## 19336 x 192

## Another analysis of which columns are most missing in the data now:
## note that there is a heavy concentration in questions d73 and d40, d25

nas2 = ebt.isnull().sum(axis = 0)
print(len(nas2[nas2>0])) ##48
print(nas2[nas2 > 0].sort_values(ascending=False))

(3872, 172)
17
d73_4        53
d73_2        53
qa4a_14      53
qa3a_9       53
d73_1        53
d73_3        53
d40abc_r1    15
d40abc_r4    15
d40abc_r3    15
d40abc_r2    15
d40b         14
d40c         14
d40c_r       14
d40b_r       14
d40a          9
d40a_r        9
d25           2
dtype: int64


Note that most of the missingness is concentrated in a few questions: here I'm going to code 999 ("Don't Know") for the missing variables that start with "q" or "s" and 0 for the other questions:

In [8]:
## could done an if/else too, but this is easy to read:

columns_fill_999 = [col for col in ebt.columns if col.startswith(('q', 's'))]
columns_fill_0 = [col for col in ebt.columns if not col.startswith(('q', 's'))]


### Fill:
ebt[columns_fill_999] = ebt[columns_fill_999].fillna(999)
ebt[columns_fill_0] = ebt[columns_fill_0].fillna(0)


In [9]:
null_check2 = ebt.isnull().sum(axis = 0)
print(null_check2[null_check2 > 0]) ## none, so we've filled our NAs

Series([], dtype: int64)


Convert string values to numeric

In [10]:
ebt = ebt.applymap(lambda x: pd.to_numeric(x, errors='ignore'))

Identify the categorical variables:

In [11]:
unique_counts = ebt.nunique() ## produces a series with unique value counts
#print(unique_counts)[1:10]

#print(unique_counts.value_counts()) ##

## assuming fewer than 20 unique values = categorical
## 20 is a large number for this, but a couple of the questions have
## a lot of possible responses
cat_cols = list(unique_counts[(unique_counts <= 15)].index)
num_cols = list(unique_counts[unique_counts > 15].index)

## qc3a is categorical
## d8 is numeric
print(num_cols)
print(cat_cols)

## Need to remove non-numeric strings
##

['d15a', 'd8', 'd9r', 'd9', 'd7', 'isocntry', 'd11', 'uniqid']
['d73_4', 'qa10_3', 'qd5_14', 'qa4a_5', 'qa2a_4', 'polintr_4', 'qa2a_5', 'd40b', 'qa6b_3', 'qa13_2', 'qa4a_4', 'd40a_r', 'd40abc_r3', 'qa4a_9', 'qa4a_13', 'qa6a_1', 'qa6a_2', 'qa2a_6', 'd70', 'qa3a_11', 'qa16', 'sd22_1', 'd40b_r', 'qa19', 'd62_4', 'polintr_1', 'qd5_6', 'qa6b_1', 'd62_2', 'netuse', 'd63', 'qa3a_17', 'qa6a_5', 'qc1a_4', 'd8r2', 'd1r2', 'd78', 'd1', 'qd5_15', 'd40c', 'd40abc_r1', 'sd22t7', 'd40abc_r4', 'qa4a_12', 'qa4a_17', 'd71_2', 'qc1a_1', 'qa1a_4', 'qa18_5', 'qa4a_18', 'qc1a_3', 'd11r1', 'qa17', 'qd5_2', 'd62_1', 'sd22t3', 'qa3a_4', 'd71_1', 'qa4a_15', 'd40abc_r2', 'qd5_1', 'qd1', 'qa2a_1', 'qd8_3', 'qa14', 'qa4a_1', 'sd22t8', 'qa1a_5', 'qa10_2', 'qa3a_3', 'qa4a_19', 'qa3a_10', 'd10', 'qa7_1', 'qa3a_2', 'qa3a_5', 'qd5_16', 'qa3a_8', 'qa3a_14', 'd73_2', 'd40c_r', 'd8r1', 'qd5_12', 'd1r1', 'sd22t6', 'qd5_9', 'd73_1', 'qa4a_16', 'd62_3', 'qa3a_16', 'qa13_1', 'qd5_10', 'qa8_1', 'qa6a_3', 'qa6b_10', 'qd5_8', 'q

Now we have a sample dataframe ("ebt") with:
- No missing data
- the categorical and numeric variables categorized (albeit according to a heuristic)


(Note that I had a change the call a bit from the video tutorial. The module name did not work, so I found a different tutorial)


In [12]:
# prompt: print the types of each column in the ebt dataframe

print(ebt.dtypes)


print("\nData types of string variables:")
string_columns = ebt.select_dtypes(include=['object']).columns
print(ebt[string_columns].dtypes)

## drop country:
## hand synthetize it later

ebt = ebt.drop('isocntry', axis=1)
#ebt['isocntry'][1:5]




d73_4           float64
qa10_3            int64
qd5_14            int64
qa4a_5            int64
qa2a_4            int64
                 ...   
mediatrust        int64
trustnom          int64
trustallm         int64
trustwebonly      int64
trusttradm        int64
Length: 172, dtype: object

Data types of string variables:
isocntry    object
dtype: object


Define the synthesizer and fit it to the dataset:
```
# This is formatted as code
```



In [13]:

#define parameters:
batch_size = 500 ## how many records to adjust the model's training at each step
epochs = 501 #number of training iterations
learning_rate  = 2e-4# determines how much the weights of the model are updated in response to
## estimated error
## beta_1 and beta_2 are regularization parameters

## CTGAN (Conditional Tabular GAN) = conditional tabular GAN

## off-the-shelf params from their quickstart guide
## for making adult census income data:

ctgan_args = ModelParameters(batch_size=batch_size,
                             lr=learning_rate,
                             betas=(0.5, 0.9))
train_args = TrainParameters(epochs=epochs)

## Train the model:

synth = RegularSynthesizer(modelname='ctgan',
                           model_parameters=ctgan_args)

synth.fit(data = ebt,
          train_arguments=train_args,
          cat_cols = cat_cols,
          num_cols = num_cols)
# Fit to dataset:


Epoch: 0 | critic_loss: -1.421227216720581 | generator_loss: 2.05618953704834
Epoch: 1 | critic_loss: -2.964867115020752 | generator_loss: 2.4976720809936523
Epoch: 2 | critic_loss: -2.483177423477173 | generator_loss: 1.3310519456863403
Epoch: 3 | critic_loss: -1.412203311920166 | generator_loss: 0.06536722183227539
Epoch: 4 | critic_loss: -0.5670332908630371 | generator_loss: -0.5608658790588379
Epoch: 5 | critic_loss: 0.04318498075008392 | generator_loss: -0.6062780618667603
Epoch: 6 | critic_loss: 0.4009580612182617 | generator_loss: -0.21152424812316895
Epoch: 7 | critic_loss: 0.2642616927623749 | generator_loss: -0.615005373954773
Epoch: 8 | critic_loss: 0.2662706673145294 | generator_loss: -0.6412056684494019
Epoch: 9 | critic_loss: 0.03358301520347595 | generator_loss: -0.536262035369873
Epoch: 10 | critic_loss: 0.19747582077980042 | generator_loss: -0.6555795669555664
Epoch: 11 | critic_loss: 0.3405742049217224 | generator_loss: -0.4654531478881836
Epoch: 12 | critic_loss: 0.4

Generate synthetic samples:

In [14]:
eb_synth = synth.sample(3000)

Save the data:

In [15]:

eb_synth.to_csv(data_path + 'synth_test.csv')

In [17]:
print(list(eb_synth.columns))

['d73_4', 'qa10_3', 'qd5_14', 'qa4a_5', 'qa2a_4', 'polintr_4', 'qa2a_5', 'd40b', 'qa6b_3', 'd15a', 'qa13_2', 'qa4a_4', 'd40a_r', 'd40abc_r3', 'qa4a_9', 'qa4a_13', 'qa6a_1', 'qa6a_2', 'qa2a_6', 'd70', 'qa3a_11', 'qa16', 'sd22_1', 'd8', 'd40b_r', 'qa19', 'd62_4', 'polintr_1', 'qd5_6', 'qa6b_1', 'd62_2', 'netuse', 'd63', 'qa3a_17', 'qa6a_5', 'qc1a_4', 'd8r2', 'd1r2', 'd78', 'd9r', 'd1', 'qd5_15', 'd40c', 'd40abc_r1', 'sd22t7', 'd40abc_r4', 'qa4a_12', 'qa4a_17', 'd71_2', 'qc1a_1', 'qa1a_4', 'qa18_5', 'qa4a_18', 'qc1a_3', 'd11r1', 'qa17', 'qd5_2', 'd62_1', 'sd22t3', 'qa3a_4', 'd71_1', 'qa4a_15', 'd9', 'd40abc_r2', 'qd5_1', 'qd1', 'qa2a_1', 'qd8_3', 'qa14', 'qa4a_1', 'sd22t8', 'qa1a_5', 'qa10_2', 'qa3a_3', 'qa4a_19', 'qa3a_10', 'd10', 'qa7_1', 'qa3a_2', 'qa3a_5', 'qd5_16', 'qa3a_8', 'qa3a_14', 'd73_2', 'd40c_r', 'd8r1', 'qd5_12', 'd1r1', 'sd22t6', 'qd5_9', 'd73_1', 'qa4a_16', 'd62_3', 'qa3a_16', 'qa13_1', 'qd5_10', 'qa8_1', 'qa6a_3', 'qa6b_10', 'qd5_8', 'qa6at', 'qa2a_3', 'qa1a_6', 'qa3a_15'